# GPU Training for XGBoost

This notebook shows usage of GPU for XGBoost training. Below we show how GPU instances can be used for the 'algorithm mode' training methods with the XGBoost container.

In this notebook we will perform XGBoost training as described [here](). See the original notebook for more details on the data. 

## Prerequisites
Ensuring the latest sagemaker sdk is installed. For a major version upgrade, there might be some apis that may get deprecated.

In [ ]:
!pip install -qU awscli boto3 sagemaker

### Setup variables and define functions

In [ ]:
%%time

import io
import os
import boto3
import sagemaker
import urllib

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-spot'
# customize to your bucket where you have would like to store the data

### Fetching the dataset

In [ ]:
%%time

# Load the dataset
FILE_DATA = 'abalone'
urllib.request.urlretrieve("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/abalone", FILE_DATA)
sagemaker.Session().upload_data(FILE_DATA, bucket=bucket, key_prefix=prefix+'/train')

### Obtaining the latest XGBoost container
We obtain the new container by specifying the framework version (1.1-1). This version specifies the upstream XGBoost framework version (1.1) and an additional SageMaker version (1). If you have an existing XGBoost workflow based on the previous (0.72) container, this would be the only change necessary to get the same workflow working with the new container.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', '1.1-1')

### Training the XGBoost model using GPU

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes few minutes.

To run our training job on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

* __container__: The container obtained in the previous step.
* __role__: Role ARN
* __hyperparameters__: A dictionary passed to the train function as hyperparameters.
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.

To run an XGBoost training job with GPU support, we have to set the parameter, `tree_method` to `gpu_hist` to utilize the full benefits of GPUs. Alongside this, we also have to use a GPU instance for our training, and for our experiment we will utilize `ml.p3.2xlarge`.

In [ ]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "silent":"0",
        "tree_method":"gpu_hist",
        "objective":"reg:squarederror",
        "num_round":"10"}

instance_type_gpu = 'ml.p3.2xlarge'
output_path = 's3://{}/{}/{}/output'.format(bucket, prefix, 'abalone-xgb')
content_type = "libsvm"

In [ ]:
import time

job_name = 'DEMO-xgboost-spot-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)

train_use_spot_instances = False
train_max_run = 60*30

estimator = sagemaker.estimator.Estimator(container, 
                                          role, 
                                          hyperparameters=hyperparameters,
                                          train_instance_count=1,
                                          train_instance_type=instance_type_gpu, 
                                          train_volume_size=10,
                                          output_path=output_path, 
                                          sagemaker_session=sagemaker.Session(),
                                          train_use_spot_instances=train_use_spot_instances, 
                                          train_max_run=train_max_run
                                         );
train_input = sagemaker.s3_input(s3_data='s3://{}/{}/{}'.format(bucket, prefix, 'train'), content_type='libsvm')
estimator.fit({'train': train_input}, job_name=job_name)

### GPU Training time
Towards the end of the job you should see two lines of output printed:

- `Training seconds: X` : This is the actual compute-time your training job spent
- `Billable seconds: Y` : This is the time you will be billed for(applicable when spot instances are used)

`Billable seconds` is the total training time using a GPU instance. 